In [ ]:
# You need to solve this assignment in four parts:

# Consolidation of similar hotel amenities
# Entity extraction of amenities from description
# Extract info - distance from nearest airport/railway station/bus stand in separate column
# Provide a way for us to test different hotel descriptions and your code should provide us the amenities on our holdout set.
# This can be you provide us a text file or notebook which runs your code on it and provide us output.

# Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
import wordninja
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering
import spacy
import random
from spacy.util import minibatch, compounding




#pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dharitripanigrahy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Read data

In [2]:
data = pd.read_csv('assignment_hotel_training.csv')
data.head()

,hotel_ameneties,property_description
0,Hot & Cold water|Credit cards accepted|Airpor...,New Soul Kiss Group of Houseboats |New Soul Ki...
1,Hot/cold Water |Parking Facility| Shower Area...,Hotel Raj International |Hotel Raj Internation...
2,Backup generator|Indoor Multi Cuisine Restaura...,Avisa Nila Beach Resort | Avisa Nila Beach Res...
3,Business centre|WiFiBathroom Toiletries |Baker...,Mayflower Hotel |Occupying a prime location on...
4,Dining Table|Geyser In Bathroom|Bathroom Toile...,The Zara Residency Hotel Located in the heart ...


In [3]:
data.shape

(8750, 2)

In [4]:
data.iloc[1,1]

'Hotel Raj International |Hotel Raj International is a budget property, located at an accessible distance from Airport, Railway Station and Bus Station.| An economy class hotel offers accommodation in hygienic environment with necessary day to day requirement. The property renders facilities like doctor on call (on request), travel desk and parking. Chargeable services include airport transfer, laundry service, taxi services and railway station transfer.| In room amenities cable television, intercom, mineral water (chargeable), iron and ironing board (on request), daily newspaper, study table and bathroom with shower area and hot/cold water toiletries.| The hotel is also an ideal place for those who come for sightseeing purposes as the hotel is very near to many sightseeing places that include Jaisalmer Fort, Bada Bagh, Salim Singh Ki Haveli, Gadsisar Lake and Tazia Tower.| Experience a comfortable and memorable stay with warm hospitality at Hotel Raj International!| '

In [5]:
data.iloc[1,0]

' Hot/cold Water |Parking Facility| Shower Area | Mineral Water |Airport Transfer|Railway Station Transfer| Bathroom Toiletries | Study Table | Intercom Facility |Laundry Service| Cable T V | Daily Newspaper |Doctor on Call| Iron|Taxi ServicesIroning Board '

# 1. Consolidating Similar aminities (Clustering)

In [6]:


# Preprocessing


sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer which is developed in recent years
stop=set(stopwords.words('english'))
sno = nltk.stem.SnowballStemmer('english')





def lemmatize_verbs(words):
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words.split():
        #print(word)
        lemma = lemmatizer.lemmatize(word)
        lemmas.append(lemma)
    return lemmas




def remove_numbers(words):
    new_words = [w for w in words if not w.isdigit()]
    return new_words

def strip_numbers(words):
    new_words = [re.sub(r'\d+', '', w) for w in words]
    #new_words = [list(map(lambda x:re.sub(r'\d+', '', x), words))]
    
    return new_words

def remove_stopwords(words):
    #print('removing stopwords')
    new_words = []
    for word in words:
        if word not in stop:
            new_words.append(word)
    return new_words

def cleanup(sent):
    words = sent.split()
    words = remove_numbers(words)
    words = strip_numbers(words)
    words = remove_stopwords(words)
    return words

def split_joinedword(sent):
    res = []
    for word in sent.split():
        words = wordninja.split(word)
        #print(words)
        if words:
            for w in words:
                res.append( w )
        else:
            res.append(word)
    #print(res)
    return ' '.join(res)
        
        
    #return ' '.join(wordninja.split(word) for word in sent.split())
    
def preprocess(sent):
    sent = ' '.join(sent.split('|'))
    sent = split_joinedword(sent)
    sents = cleanup(sent)
    return sent
    
#aminities = data['hotel_ameneties'].apply(lambda x:preprocess(x))


data['cleaened_aminities'] = data['hotel_ameneties'].apply(lambda x:' '.join(x.split('|')))
data['cleaened_aminities']  = data['cleaened_aminities'].apply(lambda x:split_joinedword(x))
data['cleaened_aminities'] = data['cleaened_aminities'].apply(lambda x: cleanup(x))
# aminities = cleanup(data['cleaened_aminities'].values)

data['cleaened_aminities'] = data['cleaened_aminities'].apply(lambda x:(sno.stem(' '.join(x).lower())))
data.head()


,hotel_ameneties,property_description,cleaened_aminities
0,Hot & Cold water|Credit cards accepted|Airpor...,New Soul Kiss Group of Houseboats |New Soul Ki...,hot & cold water credit cards accepted airport...
1,Hot/cold Water |Parking Facility| Shower Area...,Hotel Raj International |Hotel Raj Internation...,hot cold water parking facility shower area mi...
2,Backup generator|Indoor Multi Cuisine Restaura...,Avisa Nila Beach Resort | Avisa Nila Beach Res...,backup generator indoor multi cuisine restaura...
3,Business centre|WiFiBathroom Toiletries |Baker...,Mayflower Hotel |Occupying a prime location on...,business centre wi fi bathroom toiletries bake...
4,Dining Table|Geyser In Bathroom|Bathroom Toile...,The Zara Residency Hotel Located in the heart ...,dining table geyser in bathroom bathroom toile...


In [7]:
#aminities = []
aminities= list((data['hotel_ameneties'].apply(lambda x:x.split('|'))).values)
aminities=[item for sublist in aminities for item in sublist]
aminities = [x.strip().lower() for x in aminities]
aminities = [x for x in aminities if x]
aminities = list(set(aminities))


print(len(aminities))
#aminities

2287


In [8]:
# Clustering


from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
bow = count_vect.fit_transform(data['cleaened_aminities'].values)
bow.shape

(8750, 1156)

In [9]:
terms = count_vect.get_feature_names()
terms[10:50]

/Users/dharitripanigrahy/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['act',
 'activities',
 'activity',
 'acupunctur',
 'acupuncture',
 'adapters',
 'adjoining',
 'adsl',
 'adventure',
 'advised',
 'aerobics',
 'age',
 'agency',
 'agent',
 'air',
 'airline',
 'airport',
 'all',
 'allergenic',
 'allergy',
 'allow',
 'allowed',
 'ally',
 'alternative',
 'am',
 'amenities',
 'amusement',
 'and',
 'anti',
 'appli',
 'applic',
 'applicable',
 'apply',
 'appointed',
 'aqua',
 'aquarium',
 'arcad',
 'arcade',
 'archery',
 'area']

In [10]:
# Clustering

clustering = AgglomerativeClustering(n_clusters=7, affinity="cosine", linkage="average").fit(bow.toarray())
silhouette_score = metrics.silhouette_score(bow.toarray(), clustering.labels_, metric='cosine')
ss = metrics.silhouette_samples(bow.toarray(), clustering.labels_, metric='cosine') 


res = data.copy()
res['group'] = pd.Series(clustering.labels_+1)
#res['Silhouette_Score'] = pd.Series(ss)

In [11]:
print(silhouette_score)

0.3109976639366954


In [12]:
res.head()

,hotel_ameneties,property_description,cleaened_aminities,group
0,Hot & Cold water|Credit cards accepted|Airpor...,New Soul Kiss Group of Houseboats |New Soul Ki...,hot & cold water credit cards accepted airport...,1
1,Hot/cold Water |Parking Facility| Shower Area...,Hotel Raj International |Hotel Raj Internation...,hot cold water parking facility shower area mi...,1
2,Backup generator|Indoor Multi Cuisine Restaura...,Avisa Nila Beach Resort | Avisa Nila Beach Res...,backup generator indoor multi cuisine restaura...,1
3,Business centre|WiFiBathroom Toiletries |Baker...,Mayflower Hotel |Occupying a prime location on...,business centre wi fi bathroom toiletries bake...,1
4,Dining Table|Geyser In Bathroom|Bathroom Toile...,The Zara Residency Hotel Located in the heart ...,dining table geyser in bathroom bathroom toile...,1


In [13]:
res['group'].value_counts()

1    8722
7      13
3      10
2       2
6       1
4       1
5       1
Name: group, dtype: int64

In [14]:
#res[res['group']==1]

# 2. Information Extraction 

In [15]:
data.head()

,hotel_ameneties,property_description,cleaened_aminities
0,Hot & Cold water|Credit cards accepted|Airpor...,New Soul Kiss Group of Houseboats |New Soul Ki...,hot & cold water credit cards accepted airport...
1,Hot/cold Water |Parking Facility| Shower Area...,Hotel Raj International |Hotel Raj Internation...,hot cold water parking facility shower area mi...
2,Backup generator|Indoor Multi Cuisine Restaura...,Avisa Nila Beach Resort | Avisa Nila Beach Res...,backup generator indoor multi cuisine restaura...
3,Business centre|WiFiBathroom Toiletries |Baker...,Mayflower Hotel |Occupying a prime location on...,business centre wi fi bathroom toiletries bake...
4,Dining Table|Geyser In Bathroom|Bathroom Toile...,The Zara Residency Hotel Located in the heart ...,dining table geyser in bathroom bathroom toile...


In [16]:
def process_description(description):
    sents = '. '.join(description.split('|'))
    processed_token = []
    for token in sents.split():
        token = ''.join(e.lower() for e in token if e.isalnum())
        processed_token.append(token)
    #print(processed_token)
    return ' '.join(processed_token)

# d = "The Zara Residency Hotel Located in the heart of Shillong City, The Zara Residency Hotel offers tastefully decorated accommodation in 29 deluxe rooms. It is maintained with a multi-cuisine restaurant and dining hall. There is secure parking facility within the hotel premises. Each modern room at the hotel is fitted with a flat-screen TV and a study table. Mineral Water is available at a charge. Private bathrooms come with a bath tub and shower facilities. Staff at the hotel can assist with laundry services as well as travel arrangements for guests who would like to go for local sightseeing. For convenience, Airport transfers are available at a surcharge. In addition to this, The Zara Residency Hotel offers a conference hall to organize all kind of corporate and social events."
# process_description(d)
    

In [17]:
# Extract info

nlp = spacy.load('en_core_web_sm')
count = 0

def extract_info(description):
    description = process_description(description)
    #print(description)
    #if count < 11:
    dist_info = []
    doc = nlp(description)
    ents = [(e.text, e.label_) for e in doc.ents ]
    #print(ents)
    for i,ent in enumerate(ents):
        if ent[1] == 'FAC':
            try:
                if ents[i-1][1] == 'QUANTITY':
                    dist_info.append((ent[0], ents[i-1][0]))
                elif ents[i+1][1] == 'QUANTITY':
                    dist_info.append((ent[0], ents[i+1][0]))
                else:

                    dist_info.append((ent[0], 'NA'))
            except:
                pass
    return dist_info


#extract_info('Avisa Nila Beach Resort | Avisa Nila Beach Resort is located in a small town in the coastal region of Maharashtra, Malvan. It is at a distance of 132 km from Goa International Airport, 34 km from Kudal Railway Station and 27 km from Masure S.T Bus Stand.| The property has well equipped rooms with amenities like air conditioner, LED television, tea/coffee maker, safety locker and intercom facility. Each room has attached bathroom with 24 hours of hot and cold water. Hotel facilitates laundry services, generator backup and swimming pool for relaxation. The hotel has multi cuisine restaurant which serves delectable Malvani, Goan, North Indian, Chinese, Tandoor dishes.| Popular tourist attractions in Malvan are Tarkarli Beach (4 km), Tarkarli Beach (7 km), Rock Garden (9 km), Devbaug Beach (9 km), Jay Ganesh Mandir (9 km), Tsunami Island (10 km), Khapreshwar Mandir (11 km), Sindhudurg Fort (10 km), Anganewadi (21 km) and many more.| Have a pleasant and memorable stay at Avisa Nila Beach Resort!|')


data['dist_info'] = data['property_description'].apply(lambda x:extract_info(x))
    


In [18]:
data.head()

,hotel_ameneties,property_description,cleaened_aminities,dist_info
0,Hot & Cold water|Credit cards accepted|Airpor...,New Soul Kiss Group of Houseboats |New Soul Ki...,hot & cold water credit cards accepted airport...,[]
1,Hot/cold Water |Parking Facility| Shower Area...,Hotel Raj International |Hotel Raj Internation...,hot cold water parking facility shower area mi...,"[(jaisalmer fort bada, NA)]"
2,Backup generator|Indoor Multi Cuisine Restaura...,Avisa Nila Beach Resort | Avisa Nila Beach Res...,backup generator indoor multi cuisine restaura...,"[(goa international airport, 132 km), (avisa n..."
3,Business centre|WiFiBathroom Toiletries |Baker...,Mayflower Hotel |Occupying a prime location on...,business centre wi fi bathroom toiletries bake...,"[(mln road, NA), (kaziranga national park kama..."
4,Dining Table|Geyser In Bathroom|Bathroom Toile...,The Zara Residency Hotel Located in the heart ...,dining table geyser in bathroom bathroom toile...,[]


In [20]:
data.shape

(8750, 4)

In [21]:
train_df = data.iloc[:8250,:]
test_df = data.iloc[8250:, :]
print(train_df.shape)
print(test_df.shape)

(8250, 4)
(500, 4)


# 3. train NER for aminities

In [22]:
# create traindata

count = 0
TRAIN_DATA = []
for _, item in train_df.iterrows():
    ent_dict = {}
    if count < 1000:
        description = process_description(item['property_description'])
        #We will find aminities and its positions once and add to the visited items.
        visited_items = []
        entities = []
        final_entities = []
        for token in description.split():
            if token in aminities:
                for i in re.finditer(token, description):
                    if token not in visited_items:
                        
                        entity = (i.span()[0], i.span()[1], 'aminities')
                        visited_items.append(token)
                        entities.append(entity)
                        
        entities.sort(key=lambda a: a[0])
        #print('ent', entities)
        
        prev_end = 0
        for item in entities:
            if item[0] < prev_end:
                continue
            else:
                final_entities.append(item)
                prev_end = item[1]

        if len(final_entities) > 0:
            ent_dict['entities'] = final_entities
            train_item = (description, ent_dict)
            TRAIN_DATA.append(train_item)
            count+=1
            
print(TRAIN_DATA[:10])

[('new soul kiss group of houseboats  new soul kiss group of houseboats is situated in srinagar a place which truly symbolizes beauty and romance and is rightly known as the venice of the east jammu and kashmirs summer capital  literally implies city of affluence bounty guest can opt for various means to travel to srinagar srinagar airport is the clostes airport srinagar airport are the nearest airport and there are regular buses to srinagar from other major cities of the country the hotel provides various amenities like wifi internet access doctor on call facility credit card accepted laundry service dry cleaning room and housekeeping service ensures the comfortable stay of the guest long with the cleanliness of the hotel the clean and ventilated rooms in the hotel are wellmaintained and comes with amenities like television telephone hot and cold water the rooms here are perfectly designed and crafted as per the taste of guests to make their stay relaxed and comfortable guest can requ

In [23]:
n_iter = 20
def train():
    nlp = spacy.blank("en")  # create blank Language class
    print("Created blank 'en' model")
    
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
        
    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            
    nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(2.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.2,  # dropout - make it harder to memorise data
                losses=losses,
            )
        print("Losses", losses)
    return nlp

In [ ]:
#  train custom model with the training data
nlp2 = train()

Created blank 'en' model
Losses {'ner': 4837.760885646535}
Losses {'ner': 1731.8081120471813}
Losses {'ner': 1539.4208020526305}
Losses {'ner': 1431.171797600743}
Losses {'ner': 1282.3547024254412}
Losses {'ner': 1145.6030455345685}
Losses {'ner': 1092.6857087399387}
Losses {'ner': 1026.8319042718433}
Losses {'ner': 968.254744268381}


In [ ]:
for text, _ in TRAIN_DATA[:10]:
    doc = nlp2(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
test_data = test_df.iloc[:, :]['property_description']
for description in test_data:
    description = process_description(description)
    print(description)
    doc = nlp2(description)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print('________________________')